# Goodreads Data Analysis and Cleaning for Snowflake

This notebook analyzes and prepares the Goodreads dataset for Snowflake ingestion, with special focus on text reviews for AI processing.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Import our data preparation module
import sys
sys.path.append('..')
from snowflake.data_preparation import load_and_clean_data, calculate_quality_score

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Load and Examine Raw Data

In [ ]:
# Load the raw data
raw_df = pd.read_csv('../data/books.csv')

print("Dataset Shape:", raw_df.shape)
print("\nColumns:", raw_df.columns.tolist())
print("\nData Types:")
raw_df.dtypes

## 2. Data Quality Assessment

Let's examine the quality of our data before cleaning.

In [ ]:
def analyze_data_quality(df):
    quality_report = pd.DataFrame({
        'missing_values': df.isnull().sum(),
        'missing_percentage': (df.isnull().sum() / len(df) * 100).round(2),
        'unique_values': df.nunique(),
        'sample_values': df.apply(lambda x: list(x.dropna().sample(min(3, len(x.dropna()))).values))
    })
    
    return quality_report

quality_report = analyze_data_quality(raw_df)
quality_report

## 3. Text Reviews Analysis

Since we're planning to use text reviews for AI processing, let's analyze the review statistics.

In [ ]:
# Analyze review counts
plt.figure(figsize=(12, 6))
sns.histplot(data=raw_df, x='text_reviews_count', bins=50)
plt.title('Distribution of Text Review Counts')
plt.xlabel('Number of Text Reviews')
plt.ylabel('Count of Books')
plt.yscale('log')  # Log scale for better visualization
plt.show()

# Print review statistics
print("\nText Reviews Statistics:")
print(raw_df['text_reviews_count'].describe())

## 4. Clean and Transform Data

In [ ]:
# Clean the data using our preparation module
clean_df = load_and_clean_data('../data/books.csv')

# Compare before and after cleaning
print("Data Quality Scores:")
print(clean_df['data_quality_score'].describe())

# Plot quality score distribution
plt.figure(figsize=(10, 6))
sns.histplot(data=clean_df, x='data_quality_score', bins=30)
plt.title('Distribution of Data Quality Scores')
plt.xlabel('Quality Score')
plt.ylabel('Count')
plt.show()

## 5. Prepare Data Model for Snowflake

We'll create a normalized data model with separate tables for books, authors, and publishers.

In [ ]:
def create_snowflake_tables(df):
    # Create authors dimension table
    authors = (df['authors'].str.split('/')
              .explode()
              .str.strip()
              .dropna()
              .unique())
    authors_df = pd.DataFrame({
        'author_id': range(1, len(authors) + 1),
        'author_name': authors
    })
    
    # Create publishers dimension table
    publishers = df['publisher'].dropna().unique()
    publishers_df = pd.DataFrame({
        'publisher_id': range(1, len(publishers) + 1),
        'publisher_name': publishers
    })
    
    return authors_df, publishers_df

# Create dimension tables
authors_df, publishers_df = create_snowflake_tables(clean_df)

print("Number of unique authors:", len(authors_df))
print("Number of unique publishers:", len(publishers_df))

# Display sample of dimension tables
print("\nSample Authors:")
print(authors_df.head())
print("\nSample Publishers:")
print(publishers_df.head())

## 6. Export Data for Snowflake

Prepare and save the cleaned data in a format suitable for Snowflake ingestion.

In [ ]:
# Create output directory
output_dir = '../snowflake/prepared_data'
import os
os.makedirs(output_dir, exist_ok=True)

# Save the prepared files
clean_df.to_csv(f'{output_dir}/books_for_snowflake.csv', index=False)
authors_df.to_csv(f'{output_dir}/authors_for_snowflake.csv', index=False)
publishers_df.to_csv(f'{output_dir}/publishers_for_snowflake.csv', index=False)

print("Files exported successfully to:", output_dir)